# Telecom Churn Case Study

In [ ]:
#Importing Neccessary Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import RFE
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import precision_recall_curve

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

import warnings
warnings.simplefilter("ignore")
from collections import Counter
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score

In [ ]:
#Maximizing the number of rows and columns to be displayed
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [ ]:
#Loading the dataframe
data = pd.read_csv('telecom_churn_data.csv')
data.head()

## 1. Handling Missing Values

In [ ]:
data.info(verbose=True)

In [ ]:
#Looking for missing values
data.isnull().mean()

In [ ]:
#Looking at the statistical summary
data.describe()

In [ ]:
#Looking at the different types of columns
date_columns =  ['last_date_of_month_6','last_date_of_month_7','last_date_of_month_8','last_date_of_month_9','date_of_last_rech_6','date_of_last_rech_7','date_of_last_rech_8','date_of_last_rech_9','date_of_last_rech_data_6','date_of_last_rech_data_7','date_of_last_rech_data_8','date_of_last_rech_data_9']
categorical_columns = ['night_pck_user_6','night_pck_user_7','night_pck_user_8','night_pck_user_9','fb_user_6','fb_user_7','fb_user_8','fb_user_9']
id_columns = ['mobile_number', 'circle_id']
numerical_columns = [column for column in data.columns if column not in id_columns + date_columns + categorical_columns]

print('Id Columns :',len(id_columns))
print('Categorical Columns :',len(categorical_columns))
print('Numerical Columns :',len(numerical_columns))
print('Date Columns :',len(date_columns))

## Imputing Values

In [ ]:
data.head()

In [ ]:
#Handling recharge columns
rech_col = [i for i in data.columns if 'rech' in i]
data[rech_col].isnull().sum()

In [ ]:
#Looking for missing recharge columns
data[rech_col].isnull().sum()

In [ ]:
# Since recharge date and recharge amount are missing which means customer didn't recharge.
data.loc[data.total_rech_data_6.isnull() & data.date_of_last_rech_data_6.isnull(), ["total_rech_data_6", "date_of_last_rech_data_6"]].head()

In [ ]:
#list of recharge columns where we will impute missing values with zero
zero_impute = ['total_rech_data_6', 'total_rech_data_7', 'total_rech_data_8', 'total_rech_data_9',
         'av_rech_amt_data_6', 'av_rech_amt_data_7', 'av_rech_amt_data_8', 'av_rech_amt_data_9',
         'max_rech_data_6', 'max_rech_data_7', 'max_rech_data_8', 'max_rech_data_9']
data[zero_impute] = data[zero_impute].apply(lambda x:x.fillna(0))

In [ ]:
#Looking at summary statistics of the zero imputed columns
data[zero_impute].describe()

In [ ]:
#Dropping date and Id columns as they won't be useful in our analysis
print('Before dropping :',data.shape)
data = data.drop(id_columns + date_columns,axis=1)
data.head()
print('After dropping :',data.shape)

## Creating a new Category where missing values are present

In [ ]:
# Looking for unique count for categorical columns
for i in categorical_columns:
    print(data[i].value_counts())

In [ ]:
#Replacing missing categorical column values with -1
data[categorical_columns] = data[categorical_columns].apply(lambda x:x.fillna('-1'))

In [ ]:
#Checking for missing values
print(data[categorical_columns].isnull().mean())

There are no missing values present in the categorical columns

### Dropping columns having more than 65% missing values

In [ ]:
#Looking at missing value percentage by columns
round((data.isnull().sum()/len(data.index))*100,2)

In [ ]:
#Dropping the columns with more than 65% null values as it won't benefit us 

cols_with_high_missing = data.columns[data.isnull().sum()/len(data.index) > 0.65]
print('Before dropping : ',len(data.columns))
print('Columns to be dropped : ',len(cols_with_high_missing))
data = data.drop(cols_with_high_missing,axis=1)
print('After dropping : ',len(data.columns))

In [ ]:
col_with_less_na = [i for i in data.columns if data[i].isnull().mean() < 0.09 and data[i].isnull().mean() > 0]

for i in col_with_less_na:
    if i not in date_columns:
         data[i].fillna(data[i].median(),inplace=True)

In [ ]:
#Checking for missing nulls
round((data.isnull().sum()/len(data.index))*100,2)

We are now rid of missing values!

## 2. Filtering High values customers

### Calculating total Data Recharge amount

In [ ]:
data['total_data_rech_6'] = data.total_rech_data_6 * data.av_rech_amt_data_6
data['total_data_rech_7'] = data.total_rech_data_7 * data.av_rech_amt_data_7

### Calculating total Recharge amount

In [ ]:
data['amt_data_6'] = data.total_rech_amt_6 + data.total_data_rech_6
data['amt_data_7'] = data.total_rech_amt_7 + data.total_data_rech_7

### Average amount recharge done by each customer

In [ ]:
data['av_amt_data_6_7'] = (data.total_rech_amt_6 + data.total_rech_amt_7)/2

### Looking at the 70th percentile

In [ ]:
print('Recharge amount at 70th percentile:',data['av_amt_data_6_7'].quantile(0.7))

### Retaining only customers who have recharges their mobiles with more than or equal to 70th percentile amount

In [ ]:
data_exclude_70 = data.loc[data.av_amt_data_6_7 >= data.av_amt_data_6_7.quantile(0.7), :]
data_exclude_70 = data_exclude_70.reset_index(drop=True)

In [ ]:
#Deleting unwanted columns
data = data.drop(['total_data_rech_6', 'total_data_rech_7','amt_data_6', 'amt_data_7', 'av_amt_data_6_7'],axis=1)

In [ ]:
data.head()

## 3. Derive Churn

### Calculating total incoming and outgoing minutes of usage

In [ ]:
data_exclude_70['tot_inc_out_9'] = data_exclude_70.total_ic_mou_9 + data_exclude_70.total_og_mou_9

### Calculating 2g and 3g data consumption

In [ ]:
data_exclude_70['tot_internet_9'] = data_exclude_70.vol_2g_mb_9 + data_exclude_70.vol_3g_mb_9

### Creating churn variable : those who have not used either calls or internet in month of September are customers who have churned

In [ ]:
data_exclude_70['churn'] = data_exclude_70.apply(lambda x:1 if (x.tot_inc_out_9 == 0 and x.tot_internet_9 == 0) else 0, axis=1)

In [ ]:
data_exclude_70 = data_exclude_70.drop(['tot_inc_out_9','tot_internet_9'],axis=1)

### Checking churn percentage

In [ ]:
round((data_exclude_70.churn.value_counts()/len(data_exclude_70))*100,4)

### Deleting columns that belong to churn month

In [ ]:
data_exclude_70 = data_exclude_70.filter(regex='[^9]$',axis=1)

## Data Visualization

In [ ]:
#Filtering out numerical columns
data_exclude_70.info(verbose=True)

In [ ]:
categorical_columns = data_exclude_70.select_dtypes(include='object')
numerical_columns = data_exclude_70.select_dtypes(include=['int64','float64'])

In [ ]:
#Defining Boxplot function for univariate analysis 
def num_col_univariate_analysis(c):
    plt.figure(figsize=(4, 4))
    ax = sns.boxplot(y=c, data=data_exclude_70)
    plt.show()

In [ ]:
#Defining Boxplot function for bivariate analysis
def num_col_bivariate_analysis(c1,c2):
    plt.figure(figsize=(4, 4))
    ax = sns.boxplot(x=c1, y=c2, data=data_exclude_70)
    plt.show()

In [ ]:
#Univariate Analysis of Numerical Columns
for c in numerical_columns:
    num_col_univariate_analysis(c)

In [ ]:
# Univariate for categorical columns
plt.figure(figsize=(20,20))
i=0
j = 1
while i < len(categorical_columns):
    if '9' in categorical_columns[i]:
        i+=1
    else:
        plt.subplot(3,3,j)
        plt.title(categorical_columns[i])
        sns.countplot(categorical_columns[i],data=data_exclude_70)
        #print(categorical_columns[i])
        i+=1
        j+=1
plt.show()

In [ ]:
# Bivariate Analysis of Numerical Columns
for c in numerical_columns:
    num_col_bivariate_analysis('churn',c)

In [ ]:
# Visualizing the correlation between all set of usable columns
plt.figure(figsize=(30, 15))
sns.heatmap(data_exclude_70.corr(), cmap="YlGnBu")

In [ ]:
i=0
plt.figure(figsize=(200,200))
while i < len(numerical_columns):
    plt.subplot(13,13,i+1)
    plt.title(numerical_columns[i])
    sns.boxplot(y=numerical_columns[i],data=data_exclude_70_out)
    i+=1
plt.show()

## 5. Derived Variable

In [ ]:
data_exclude_70['arpu_diff'] = data_exclude_70.arpu_8 - ((data_exclude_70.arpu_6 + data_exclude_70.arpu_7)/2)
data_exclude_70['onnet_mou_diff'] = data_exclude_70.onnet_mou_8 - ((data_exclude_70.onnet_mou_6 + data_exclude_70.onnet_mou_7)/2)
data_exclude_70['offnet_mou_diff'] = data_exclude_70.offnet_mou_8 - ((data_exclude_70.offnet_mou_6 + data_exclude_70.offnet_mou_7)/2)
data_exclude_70['roam_ic_mou_diff'] = data_exclude_70.roam_ic_mou_8 - ((data_exclude_70.roam_ic_mou_6 + data_exclude_70.roam_ic_mou_7)/2)
data_exclude_70['roam_og_mou_diff'] = data_exclude_70.roam_og_mou_8 - ((data_exclude_70.roam_og_mou_6 + data_exclude_70.roam_og_mou_7)/2)
data_exclude_70['loc_og_mou_diff'] = data_exclude_70.loc_og_mou_8 - ((data_exclude_70.loc_og_mou_6 + data_exclude_70.loc_og_mou_7)/2)
data_exclude_70['std_og_mou_diff'] = data_exclude_70.std_og_mou_8 - ((data_exclude_70.std_og_mou_6 + data_exclude_70.std_og_mou_7)/2)
data_exclude_70['isd_og_mou_diff'] = data_exclude_70.isd_og_mou_8 - ((data_exclude_70.isd_og_mou_6 + data_exclude_70.isd_og_mou_7)/2)
data_exclude_70['spl_og_mou_diff'] = data_exclude_70.spl_og_mou_8 - ((data_exclude_70.spl_og_mou_6 + data_exclude_70.spl_og_mou_7)/2)
data_exclude_70['total_og_mou_diff'] = data_exclude_70.total_og_mou_8 - ((data_exclude_70.total_og_mou_6 + data_exclude_70.total_og_mou_7)/2)
data_exclude_70['loc_ic_mou_diff'] = data_exclude_70.loc_ic_mou_8 - ((data_exclude_70.loc_ic_mou_6 + data_exclude_70.loc_ic_mou_7)/2)
data_exclude_70['std_ic_mou_diff'] = data_exclude_70.std_ic_mou_8 - ((data_exclude_70.std_ic_mou_6 + data_exclude_70.std_ic_mou_7)/2)
data_exclude_70['isd_ic_mou_diff'] = data_exclude_70.isd_ic_mou_8 - ((data_exclude_70.isd_ic_mou_6 + data_exclude_70.isd_ic_mou_7)/2)
data_exclude_70['spl_ic_mou_diff'] = data_exclude_70.spl_ic_mou_8 - ((data_exclude_70.spl_ic_mou_6 + data_exclude_70.spl_ic_mou_7)/2)
data_exclude_70['total_ic_mou_diff'] = data_exclude_70.total_ic_mou_8 - ((data_exclude_70.total_ic_mou_6 + data_exclude_70.total_ic_mou_7)/2)
data_exclude_70['total_rech_num_diff'] = data_exclude_70.total_rech_num_8 - ((data_exclude_70.total_rech_num_6 + data_exclude_70.total_rech_num_7)/2)
data_exclude_70['total_rech_amt_diff'] = data_exclude_70.total_rech_amt_8 - ((data_exclude_70.total_rech_amt_6 + data_exclude_70.total_rech_amt_7)/2)
data_exclude_70['max_rech_amt_diff'] = data_exclude_70.max_rech_amt_8 - ((data_exclude_70.max_rech_amt_6 + data_exclude_70.max_rech_amt_7)/2)
data_exclude_70['total_rech_data_diff'] = data_exclude_70.total_rech_data_8 - ((data_exclude_70.total_rech_data_6 + data_exclude_70.total_rech_data_7)/2)
data_exclude_70['max_rech_data_diff'] = data_exclude_70.max_rech_data_8 - ((data_exclude_70.max_rech_data_6 + data_exclude_70.max_rech_data_7)/2)
data_exclude_70['av_rech_amt_data_diff'] = data_exclude_70.av_rech_amt_data_8 - ((data_exclude_70.av_rech_amt_data_6 + data_exclude_70.av_rech_amt_data_7)/2)
data_exclude_70['vol_2g_mb_diff'] = data_exclude_70.vol_2g_mb_8 - ((data_exclude_70.vol_2g_mb_6 + data_exclude_70.vol_2g_mb_7)/2)
data_exclude_70['vol_3g_mb_diff'] = data_exclude_70.vol_3g_mb_8 - ((data_exclude_70.vol_3g_mb_6 + data_exclude_70.vol_3g_mb_7)/2)

## 6. Handling Outliers

In [ ]:
#Excluding the Outliers in the numerical columns

data_exclude_70_out = data_exclude_70.copy()
for i in numerical_columns:
    UR = data_exclude_70_out[i].quantile(0.999)
    LR = data_exclude_70_out[i].quantile(0.001)
    data_exclude_70_out = data_exclude_70_out[(data_exclude_70_out[i] >= LR) & (data_exclude_70_out[i] <= UR)]
data_exclude_70_out.shape

## 7. Creating dummy variable 

In [ ]:
categorical_columns =['night_pck_user_6',
 'night_pck_user_7',
 'night_pck_user_8',
 'fb_user_6',
 'fb_user_7',
 'fb_user_8',
 ]

data_exclude_dummy = pd.get_dummies(data_exclude_70_out[categorical_columns])
data_exclude_70_drop = data_exclude_70_out.drop(categorical_columns,axis=1)

In [ ]:
data_with_dummy = pd.concat([data_exclude_70_drop,data_exclude_dummy],axis=1)

In [ ]:
# change churn to numeric
data_with_dummy['churn'] = pd.to_numeric(data_with_dummy['churn'])

##  8. Performing Train-Test Split

In [ ]:
#Divide data into train test split

from sklearn.model_selection import train_test_split
X = data_with_dummy.drop("churn", axis = 1)
y = data_with_dummy.churn
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 4, stratify = y)

In [ ]:
#Print shapes of train and test sets
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

## 9 .Handling Imbalance

In [ ]:
#Calculating the churn rate
round((data_exclude_70.churn.value_counts()/len(data_exclude_70))*100,4)

In [ ]:
from imblearn.over_sampling import SMOTE
smt = SMOTE(random_state=45, k_neighbors=5)
X_train_resampled_smt, y_train_resampled_smt = smt.fit_resample(X_train, y_train)
X_train_resampled_smt.head


## 10. Scaling 

In [ ]:
# Scaling the columns using StandardScaler on train dataset
from sklearn.preprocessing import StandardScaler
col = [i for i in X_train_resampled_smt.columns if i not in ['night_pck_user_6','night_pck_user_7','night_pck_user_8','fb_user_6','fb_user_7','fb_user_8']]
scaler = StandardScaler()
X_train_resampled_smt[col] = scaler.fit_transform(X_train_resampled_smt[col])
X_train_resampled_smt.head()

In [ ]:
# Scaling the columns using StandardScaler on test dataset
from sklearn.preprocessing import StandardScaler
X_test_scaled = X_test.copy()
col = [i for i in X_test.columns if i not in ['night_pck_user_6','night_pck_user_7','night_pck_user_8','fb_user_6','fb_user_7','fb_user_8']]
X_test_scaled[col] = scaler.transform(X_test_scaled[col])
X_test_scaled.head()

## 11. Modelling

### Model I : Logistic Regression without PCA

In [ ]:
#Creating the LogisticRegression object and predicting the results on test 
from sklearn.linear_model import LogisticRegression
lreg = LogisticRegression()
lreg.fit(X_train_resampled_smt, y_train_resampled_smt)
y_pred = lreg.predict(X_test_scaled)

In [ ]:
#Selecting 15 features that describes the dependent variable the most
logreg = LogisticRegression(random_state=100)
rfe = RFE(logreg, 15)             # running RFE with 15 variables as output
rfe = rfe.fit(X_train_resampled_smt, y_train_resampled_smt)

In [ ]:
#Printing list of columns selected by the RFE
rfe_selected_columns = pd.DataFrame(X_train_resampled_smt).columns[rfe.support_]
rfe_selected_columns

In [ ]:
#Looking at the statistical summary of the model created
feature_list = ['onnet_mou_8', 'std_og_t2m_mou_8', 'total_og_mou_8',
       'total_rech_data_6', 'total_rech_data_7', 'count_rech_2g_6',
       'count_rech_2g_7', 'count_rech_3g_6', 'count_rech_3g_7', 'monthly_2g_7',
       'sachet_2g_7', 'monthly_3g_6', 'monthly_3g_7', 'sachet_3g_6',
       'sachet_3g_7']
X_train_resampled_smt_sm = sm.add_constant(X_train_resampled_smt[rfe_selected_columns])
logm = sm.GLM(y_train_resampled_smt,X_train_resampled_smt_sm, family = sm.families.Binomial())
logm.fit().summary()

In [ ]:
#Removing feature with high pvalues
feature_list.remove('sachet_3g_6')
X_train_resampled_smt_sm = sm.add_constant(X_train_resampled_smt[feature_list])
logm = sm.GLM(y_train_resampled_smt,X_train_resampled_smt_sm, family = sm.families.Binomial())
logm.fit().summary()

In [ ]:
#Removing feature with high pvalues
feature_list.remove('sachet_3g_7')
X_train_resampled_smt_sm = sm.add_constant(X_train_resampled_smt[feature_list])
logm = sm.GLM(y_train_resampled_smt,X_train_resampled_smt_sm, family = sm.families.Binomial())
logm.fit().summary()

In [ ]:
#Removing feature with high pvalues
feature_list.remove('sachet_2g_7')
X_train_resampled_smt_sm = sm.add_constant(X_train_resampled_smt[feature_list])
logm = sm.GLM(y_train_resampled_smt,X_train_resampled_smt_sm, family = sm.families.Binomial())
logm.fit().summary()

In [ ]:
#Removing feature with high pvalues
feature_list.remove('count_rech_3g_7')
X_train_resampled_smt_sm = sm.add_constant(X_train_resampled_smt[feature_list])
logm = sm.GLM(y_train_resampled_smt,X_train_resampled_smt_sm, family = sm.families.Binomial())
logm.fit().summary()

In [ ]:
#Removing feature with high pvalues
feature_list.remove('count_rech_2g_6')
X_train_resampled_smt_sm = sm.add_constant(X_train_resampled_smt[feature_list])
logm = sm.GLM(y_train_resampled_smt,X_train_resampled_smt_sm, family = sm.families.Binomial())
res = logm.fit().summary()
res

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif = pd.DataFrame()
vif['Features'] = X_train_resampled_smt[feature_list].columns
vif['VIF'] = [variance_inflation_factor(X_train_resampled_smt[feature_list].values, i) for i in range(X_train_resampled_smt[feature_list].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
#Removing feature with high VIF
feature_list.remove('total_rech_data_7')
X_train_resampled_smt_sm = sm.add_constant(X_train_resampled_smt[feature_list])
logm = sm.GLM(y_train_resampled_smt,X_train_resampled_smt_sm, family = sm.families.Binomial())
res = logm.fit()
res.summary()

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif = pd.DataFrame()
vif['Features'] = X_train_resampled_smt[feature_list].columns
vif['VIF'] = [variance_inflation_factor(X_train_resampled_smt[feature_list].values, i) for i in range(X_train_resampled_smt[feature_list].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
#Removing feature with high VIF
feature_list.remove('total_og_mou_8')
X_train_resampled_smt_sm = sm.add_constant(X_train_resampled_smt[feature_list])
logm = sm.GLM(y_train_resampled_smt,X_train_resampled_smt_sm, family = sm.families.Binomial())
res = logm.fit()
res.summary()

In [ ]:
#Removing feature with high VIF
feature_list.remove('total_rech_data_6')
X_train_resampled_smt_sm = sm.add_constant(X_train_resampled_smt[feature_list])
logm = sm.GLM(y_train_resampled_smt,X_train_resampled_smt_sm, family = sm.families.Binomial())
res = logm.fit()
res.summary()

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif = pd.DataFrame()
vif['Features'] = X_train_resampled_smt[feature_list].columns
vif['VIF'] = [variance_inflation_factor(X_train_resampled_smt[feature_list].values, i) for i in range(X_train_resampled_smt[feature_list].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
y_train_resampled_smt_pred = res.predict(X_train_resampled_smt_sm).values.reshape(-1)

In [ ]:
#Predicting the probability of churn for each customer
y_train_pred_final = pd.DataFrame({'ActualChurn':y_train_resampled_smt.values, 'ChurnProbability':y_train_resampled_smt_pred})
y_train_pred_final['Predicted'] = y_train_pred_final.ChurnProbability.map(lambda x: 1 if x > 0.5 else 0)
y_train_pred_final.head()

In [ ]:
# check the overall accuracy.
print(metrics.accuracy_score(y_train_pred_final.ActualChurn, y_train_pred_final.Predicted))

In [ ]:
confusion = confusion_matrix(y_train_pred_final.ActualChurn, y_train_pred_final.Predicted)

In [ ]:
print ('Accuracy: ', accuracy_score(y_train_pred_final.ActualChurn, y_train_pred_final.Predicted))
print ('F1 score: ', f1_score(y_train_pred_final.ActualChurn, y_train_pred_final.Predicted))
print ('Recall: ', recall_score(y_train_pred_final.ActualChurn, y_train_pred_final.Predicted))
print ('Precision: ', precision_score(y_train_pred_final.ActualChurn, y_train_pred_final.Predicted))
print ('\n clasification report:\n', classification_report(y_train_pred_final.ActualChurn, y_train_pred_final.Predicted))
print ('\n confussion matrix:\n',confusion_matrix(y_train_pred_final.ActualChurn, y_train_pred_final.Predicted))

In [ ]:
def draw_roc( actual, probs ):
    fpr, tpr, thresholds = metrics.roc_curve(actual, probs, drop_intermediate = False)
    auc_score = metrics.roc_auc_score(actual, probs)
    plt.figure(figsize=(7,7))
    plt.plot( fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score )
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic Curve')
    plt.legend(loc="lower right")
    plt.show()

    return None
draw_roc(y_train_pred_final.ActualChurn, y_train_pred_final.ChurnProbability)

In [ ]:
# Create columns with different probability cutoffs 
numbers = [float(x)/10 for x in range(10)]

for i in numbers:
    y_train_pred_final[i]= y_train_pred_final.ChurnProbability.map(lambda x: 1 if x > i else 0)
    
y_train_pred_final.head()

In [ ]:
# Calculate accuracy sensitivity and specificity for various probability cutoffs.
cutoff_df = pd.DataFrame( columns = ['CutOff_Probability','Accuracy','Sensitivity','Specificity'])

# TP = confusion[1,1] # true positive 
# TN = confusion[0,0] # true negatives
# FP = confusion[0,1] # false positives
# FN = confusion[1,0] # false negatives

num = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for i in num:
    cm1 = metrics.confusion_matrix(y_train_pred_final.ActualChurn, y_train_pred_final[i])
    total1=sum(sum(cm1))
    Accuracy = (cm1[0,0]+cm1[1,1])/total1
    
    Specificity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    Sensitivity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    cutoff_df.loc[i] =[ i ,Accuracy,Sensitivity,Specificity]
    
print(cutoff_df)

In [ ]:
# Plot accuracy sensitivity and specificity for various probabilities.
plt.figure(figsize=(8, 6))
plt.plot(cutoff_df['CutOff_Probability'], cutoff_df['Accuracy'], label='Accuracy')
plt.plot(cutoff_df['CutOff_Probability'], cutoff_df['Sensitivity'], label='Sensitivity')
plt.plot(cutoff_df['CutOff_Probability'], cutoff_df['Specificity'], label='Specificity')
plt.xlabel('Probability Cutoff')
plt.title('Probability Cutoff vs Accuracy, Sensitivity and Specificity')
plt.legend()
plt.show()

In [ ]:
# Assign label based on optimul cutoff probability
y_train_pred_final['final_predicted'] = y_train_pred_final.ChurnProbability.map( lambda x: 1 if x > .58 else 0)
y_train_pred_final.head()

In [ ]:
# check the overall accuracy.
metrics.accuracy_score(y_train_pred_final.ActualChurn, y_train_pred_final.final_predicted)

In [ ]:
print ('Accuracy: ', accuracy_score(y_train_pred_final.ActualChurn, y_train_pred_final.Predicted))
print ('F1 score: ', f1_score(y_train_pred_final.ActualChurn, y_train_pred_final.Predicted))
print ('Recall: ', recall_score(y_train_pred_final.ActualChurn, y_train_pred_final.Predicted))
print ('Precision: ', precision_score(y_train_pred_final.ActualChurn, y_train_pred_final.Predicted))
print ('\n clasification report:\n', classification_report(y_train_pred_final.ActualChurn, y_train_pred_final.Predicted))
print ('\n confussion matrix:\n',confusion_matrix(y_train_pred_final.ActualChurn, y_train_pred_final.Predicted))

In [ ]:
#Predicting on test dataset
X_test_new = X_test_scaled[feature_list]
X_test_sm = sm.add_constant(X_test_new)

y_test_pred = res.predict(X_test_sm).values.reshape(-1)

# Converting y_pred to a dataframe which is an array
y_pred_1 = pd.DataFrame(y_test_pred)

# Converting y_test to dataframe
y_test_df = pd.DataFrame(y_test)

In [ ]:
# Removing index for both dataframes to append them side by side 
y_pred_1.reset_index(drop=True, inplace=True)
y_test_df.reset_index(drop=True, inplace=True)

# Appending y_test_df and y_pred_1
y_pred_final = pd.concat([y_test_df, y_pred_1],axis=1)
y_pred_final.head()

In [ ]:
# Renaming the column 
y_pred_final= y_pred_final.rename(columns={ 0 : 'Churn_Prob'})
y_pred_final['final_predicted'] = y_pred_final.Churn_Prob.map(lambda x: 1 if x > 0.58 else 0)
y_pred_final.head()

In [ ]:
# Let's check the overall accuracy.
metrics.accuracy_score(y_pred_final.churn, y_pred_final.final_predicted)

In [ ]:
print('Test Dataset :')
print ('Accuracy: ', accuracy_score(y_pred_final.churn, y_pred_final.final_predicted))
print ('F1 score: ', f1_score(y_pred_final.churn, y_pred_final.final_predicted))
print ('Recall: ', recall_score(y_pred_final.churn, y_pred_final.final_predicted))
print ('Precision: ', precision_score(y_pred_final.churn, y_pred_final.final_predicted))
print ('\n clasification report:\n', classification_report(y_pred_final.churn, y_pred_final.final_predicted))
print ('\n confussion matrix:\n',confusion_matrix(y_pred_final.churn, y_pred_final.final_predicted))

We could see that the model performance and the metrics were low. So we will further implement PCA with Logistic Regression and see what results we get.

## Model II : PCA and Logistic Regression

In [ ]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

# apply pca to train data
pca = Pipeline([('pca', PCA())])

In [ ]:
#Fitting the model on training dataset
pca.fit(X_train_resampled_smt)
churn_pca = pca.fit_transform(X_train_resampled_smt)

In [ ]:
# extract pca model from pipeline
pca = pca.named_steps['pca']

# look at explainded variance of PCA components
print(pd.Series(np.round(pca.explained_variance_ratio_.cumsum(), 4)*100))

In [ ]:
var_cumu = np.cumsum(pca.explained_variance_ratio_)
fig = plt.figure(figsize=[12,8])
plt.vlines(x=15, ymax=1, ymin=0, colors="r", linestyles="--")
plt.hlines(y=0.95, xmax=30, xmin=0, colors="g", linestyles="--")
plt.plot(var_cumu)
plt.ylabel("Cumulative variance explained")
plt.show()

In [ ]:
# create pipeline
PCA_VARS = 60
steps = [("pca", PCA(n_components=PCA_VARS)),
         ("logistic", LogisticRegression(class_weight='balanced'))
        ]
pipeline = Pipeline(steps)

In [ ]:
# fit model
pipeline.fit(X_train_resampled_smt, y_train_resampled_smt)

# check score on train data
pipeline.score(X_train_resampled_smt, y_train_resampled_smt)

In [ ]:
# predict churn on test data
y_pred = pipeline.predict(X_test_scaled)

# create onfusion matrix
confusion = confusion_matrix(y_test, y_pred)
print(confusion)

# check sensitivity and specificity
TP = confusion[1,1] # true positive 
TN = confusion[0,0] # true negatives
FP = confusion[0,1] # false positives
FN = confusion[1,0] # false negatives

# Calculate sensitivity
sensitivity = TP / float(TP+FN)

# Calculate specificity
specificity = TN / float(TN+FP)
print("Sensitivity: \t", round(sensitivity, 2), "\n", "Specificity: \t", round(specificity, 2), sep='')

# check area under curve
y_pred_prob = pipeline.predict_proba(X_test)[:, 1]
print("AUC:    \t", round(roc_auc_score(y_test, y_pred_prob),2))

In [ ]:
print('Test Dataset :')
print ('Accuracy: ', accuracy_score(y_test, y_pred))
print ('F1 score: ', f1_score(y_test, y_pred))
print ('Recall: ', recall_score(y_test, y_pred))
print ('Precision: ', precision_score(y_test, y_pred))
print ('\n clasification report:\n', classification_report(y_test,y_pred))
print ('\n confussion matrix:\n',confusion_matrix(y_test, y_pred))
confusion = confusion_matrix(y_test, y_pred)

#### Hyperparameter tuning - PCA and logistic Regression

In [ ]:
# class imbalance
y_train.value_counts()/y_train.shape

In [ ]:
# PCA
pca = PCA(random_state=42)

# logistic regression - the class weight is used to handle class imbalance - it adjusts the cost function
logistic = LogisticRegression()#class_weight={0:0.1, 1: 0.9})

# create pipeline
steps = [ 
         ("pca", pca),
         ("logistic", logistic)
        ]

# compile pipeline
pca_logistic = Pipeline(steps)

# hyperparameter space
params = {'pca__n_components': [50,60, 80,90,100], 'logistic__C': [0.001,0.01,0.1, 0.5, 1, 2, 3, 4, 5, 10], 'logistic__penalty': ['l1', 'l2']}

# create 5 folds
folds = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 4)

# create gridsearch object
model = GridSearchCV(estimator=pca_logistic, cv=folds, param_grid=params, scoring='roc_auc', n_jobs=-1, verbose=1)

In [ ]:
# fit model
model.fit(X_train_resampled_smt, y_train_resampled_smt)

In [ ]:
# cross validation results
pd.DataFrame(model.cv_results_)

In [ ]:
# print best hyperparameters
print("Best AUC: ", model.best_score_)
print("Best hyperparameters: ", model.best_params_)

In [ ]:
# PCA
pca = PCA(random_state=42)

# logistic regression - the class weight is used to handle class imbalance - it adjusts the cost function
logistic = LogisticRegression()#class_weight={0:0.1, 1: 0.9})

# create pipeline
steps = [ 
         ("pca", pca),
         ("logistic", logistic)
        ]

# compile pipeline
pca_logistic = Pipeline(steps)

# hyperparameter space
params = {'pca__n_components': [100], 'logistic__C': [10], 'logistic__penalty': ['l2']}

# create 5 folds
folds = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 4)

# create gridsearch object
model = GridSearchCV(estimator=pca_logistic, cv=folds, param_grid=params, scoring='roc_auc', n_jobs=-1, verbose=1)

In [ ]:
# fit model
model.fit(X_train_resampled_smt, y_train_resampled_smt)

In [ ]:
# Evaluating on test data
y_pred = pipeline.predict(X_test_scaled)

# create onfusion matrix
confusion = confusion_matrix(y_test, y_pred)
print(confusion)

# check sensitivity and specificity
TP = confusion[1,1] # true positive 
TN = confusion[0,0] # true negatives
FP = confusion[0,1] # false positives
FN = confusion[1,0] # false negatives

# Calculate sensitivity
sensitivity = TP / float(TP+FN)

# Calculate specificity
specificity = TN / float(TN+FP)
print("Sensitivity: \t", round(sensitivity, 2), "\n", "Specificity: \t", round(specificity, 2), sep='')

# check area under curve
y_pred_prob = pipeline.predict_proba(X_test)[:, 1]
print("AUC:    \t", round(roc_auc_score(y_test, y_pred_prob),2))

In [ ]:
print('Test Dataset :')
print ('Accuracy: ', accuracy_score(y_test, y_pred))
print ('F1 score: ', f1_score(y_test, y_pred))
print ('Recall: ', recall_score(y_test, y_pred))
print ('Precision: ', precision_score(y_test, y_pred))
print ('\n clasification report:\n', classification_report(y_test,y_pred))
print ('\n confussion matrix:\n',confusion_matrix(y_test, y_pred))
confusion = confusion_matrix(y_test, y_pred)

### Model III : Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
# PCA
pca = PCA(random_state=42)
decision_tree = DecisionTreeClassifier()#class_weight={0:0.1, 1: 0.9})

# create pipeline
steps = [ 
         ("pca", pca),
         ("decisiontreeclassifier", decision_tree)
        ]

# compile pipeline
pca_dt = Pipeline(steps)

# hyperparameter space
params = {'pca__n_components': [50,60,80,90,100,120],'decisiontreeclassifier__max_depth':[3,4,5],
              'decisiontreeclassifier__min_samples_leaf':[10,20,30]}

# create 5 folds
folds = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 4)

# create gridsearch object
model = GridSearchCV(estimator=pca_dt, cv=folds, param_grid=params, n_jobs=-1, verbose=1)

In [ ]:
model.fit(X_train_resampled_smt, y_train_resampled_smt)

In [ ]:
# print best hyperparameters
print("Best AUC: ", model.best_score_)
print("Best hyperparameters: ", model.best_params_)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
# PCA
pca = PCA(random_state=42)
decision_tree = DecisionTreeClassifier()#class_weight={0:0.1, 1: 0.9})

# create pipeline
steps = [ 
         ("pca", pca),
         ("decisiontreeclassifier", decision_tree)
        ]

# compile pipeline
pca_dt = Pipeline(steps)

# hyperparameter space
params = {'pca__n_components': [120],'decisiontreeclassifier__max_depth':[5],
              'decisiontreeclassifier__min_samples_leaf':[10]}

# create 5 folds
folds = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 4)

# create gridsearch object
model = GridSearchCV(estimator=pca_dt, cv=folds, param_grid=params, n_jobs=-1, verbose=1)

In [ ]:
# Evaluating on test data

# predict churn on test data
y_pred = pipeline.predict(X_test_scaled)

# create onfusion matrix
confusion = confusion_matrix(y_test, y_pred)
print(confusion)

# check sensitivity and specificity
TP = confusion[1,1] # true positive 
TN = confusion[0,0] # true negatives
FP = confusion[0,1] # false positives
FN = confusion[1,0] # false negatives

# Calculate sensitivity
sensitivity = TP / float(TP+FN)

# Calculate specificity
specificity = TN / float(TN+FP)
print("Sensitivity: \t", round(sensitivity, 2), "\n", "Specificity: \t", round(specificity, 2), sep='')

# check area under curve
y_pred_prob = pipeline.predict_proba(X_test)[:, 1]
print("AUC:    \t", round(roc_auc_score(y_test, y_pred_prob),2))

In [ ]:
print('Test Dataset :')
print ('Accuracy: ', accuracy_score(y_test, y_pred))
print ('F1 score: ', f1_score(y_test, y_pred))
print ('Recall: ', recall_score(y_test, y_pred))
print ('Precision: ', precision_score(y_test, y_pred))
print ('\n clasification report:\n', classification_report(y_test,y_pred))
print ('\n confussion matrix:\n',confusion_matrix(y_test, y_pred))
confusion = confusion_matrix(y_test, y_pred)

### Model IV : Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# PCA
pca = PCA(random_state=42)
random_forest = RandomForestClassifier()#class_weight={0:0.1, 1: 0.9})

# create pipeline
steps = [ 
         ("pca", pca),
         ("RandomForestClassifier", random_forest)
        ]

# compile pipeline
pca_dt = Pipeline(steps)

# hyperparameter space
params = {'pca__n_components': [60,80,100,120],'RandomForestClassifier__max_depth':[3,4,5],
              'RandomForestClassifier__min_samples_leaf':[10,20,30],'RandomForestClassifier__n_estimators':[10,20,30,40]}

# create 5 folds
folds = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 4)

# create gridsearch object
model = GridSearchCV(estimator=pca_dt, cv=folds, param_grid=params, n_jobs=-1, verbose=1)

In [ ]:
model.fit(X_train_resampled_smt, y_train_resampled_smt)

In [ ]:
# print best hyperparameters
print("Best AUC: ", model.best_score_)
print("Best hyperparameters: ", model.best_params_)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# PCA
pca = PCA(random_state=42)
random_forest = RandomForestClassifier()#class_weight={0:0.1, 1: 0.9})

# create pipeline
steps = [ 
         ("pca", pca),
         ("RandomForestClassifier", random_forest)
        ]

# compile pipeline
pca_dt = Pipeline(steps)

# hyperparameter space
params = {'pca__n_components': [80],'RandomForestClassifier__max_depth':[5],
              'RandomForestClassifier__min_samples_leaf':[20],'RandomForestClassifier__n_estimators':[20]}

# create 5 folds
folds = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 4)

# create gridsearch object
model = GridSearchCV(estimator=pca_dt, cv=folds, param_grid=params, n_jobs=-1, verbose=1)

In [ ]:
# Evaluating on test data
# predict churn on test data
y_pred = pipeline.predict(X_test_scaled)

# create onfusion matrix
confusion = confusion_matrix(y_test, y_pred)
print(confusion)

# check sensitivity and specificity
TP = confusion[1,1] # true positive 
TN = confusion[0,0] # true negatives
FP = confusion[0,1] # false positives
FN = confusion[1,0] # false negatives

# Calculate sensitivity
sensitivity = TP / float(TP+FN)

# Calculate specificity
specificity = TN / float(TN+FP)
print("Sensitivity: \t", round(sensitivity, 2), "\n", "Specificity: \t", round(specificity, 2), sep='')

# check area under curve
y_pred_prob = pipeline.predict_proba(X_test_scaled)[:, 1]
print("AUC:    \t", round(roc_auc_score(y_test, y_pred_prob),2))

In [ ]:
print('Test Dataset :')
print ('Accuracy: ', accuracy_score(y_test, y_pred))
print ('F1 score: ', f1_score(y_test, y_pred))
print ('Recall: ', recall_score(y_test, y_pred))
print ('Precision: ', precision_score(y_test, y_pred))
print ('\n clasification report:\n', classification_report(y_test,y_pred))
print ('\n confussion matrix:\n',confusion_matrix(y_test, y_pred))
confusion = confusion_matrix(y_test, y_pred)

In [ ]:
# run a random forest model on train data
max_features = int(round(np.sqrt(X_train.shape[1])))    # number of variables to consider to split each node
print(max_features)

rf_model = RandomForestClassifier(n_estimators=100, max_features=max_features, class_weight={0:0.1, 1: 0.9}, oob_score=True, random_state=4, verbose=1)

In [ ]:
# fit model
rf_model.fit(X_train, y_train)

In [ ]:
# OOB score
rf_model.oob_score_

In [ ]:
# predict churn on test data
y_pred = rf_model.predict(X_test)

#Confusion Matrix
TP = confusion[1,1] # true positive 
TN = confusion[0,0] # true negatives
FP = confusion[0,1] # false positives
FN = confusion[1,0] # false negatives

# Calculate sensitivity
print('Sensitivity:',TP / float(TP+FN))

# Calculate specificity
print('Specificity:',TN / float(TN+FP))

# Calculate false postive rate - predicting Converted when customer does not have Converted
print('False Positive Rate:',FP/ float(TN+FP))

# positive predictive value 
print('Positive Predictive Value:',TP / float(TP+FP))

# Negative predictive value
print('Negative Predictive Value:',TN / float(TN+ FN))

In [ ]:
print ('Test Dataset :')
print ('Accuracy: ', accuracy_score(y_test, y_pred))
print ('F1 score: ', f1_score(y_test, y_pred))
print ('Recall: ', recall_score(y_test, y_pred))
print ('Precision: ', precision_score(y_test, y_pred))
print ('\n clasification report:\n', classification_report(y_test,y_pred))
print ('\n confussion matrix:\n',confusion_matrix(y_test, y_pred))
confusion = confusion_matrix(y_test, y_pred)

### Feature Importances

In [ ]:
# predictors
features = data_exclude_70.drop('churn', axis=1).columns

# feature_importance
importance = rf_model.feature_importances_

# create dataframe
feature_importance = pd.DataFrame({'variables': features, 'importance_percentage': importance*100})
feature_importance = feature_importance[['variables', 'importance_percentage']]

# sort features
feature_importance = feature_importance.sort_values('importance_percentage', ascending=False).reset_index(drop=True)
print("Sum of importance=", feature_importance.importance_percentage.sum())
feature_importance

In [ ]:
# extract top 'n' features
top_n = 30
top_features = feature_importance.variables[0:top_n]

In [ ]:
# plot feature correlation
import seaborn as sns
plt.rcParams["figure.figsize"] =(10,10)
mycmap = sns.diverging_palette(199, 359, s=99, center="light", as_cmap=True)
sns.heatmap(data=X_train[top_features].corr(), center=0.0, cmap=mycmap)

In [ ]:
top_features = ['total_ic_mou_8', 'total_rech_amt_diff', 'total_og_mou_8', 'arpu_8', 'roam_ic_mou_8', 'roam_og_mou_8', 
                'std_ic_mou_8', 'av_rech_amt_data_8', 'std_og_mou_8']
X_train = X_train[top_features]
X_test = X_test[top_features]

## Extracting intercept and slope from logistic regression

In [ ]:
logistic_model = model.best_estimator_.named_steps['logistic']

In [ ]:
# intercept
intercept_df = pd.DataFrame(logistic_model.intercept_.reshape((1,1)), columns = ['intercept'])

In [ ]:
# coefficients
coefficients = logistic_model.coef_.reshape((9, 1)).tolist()
coefficients = [val for sublist in coefficients for val in sublist]
coefficients = [round(coefficient, 3) for coefficient in coefficients]

logistic_features = list(X_train.columns)
coefficients_df = pd.DataFrame(logistic_model.coef_, columns=logistic_features)

In [ ]:
# concatenate dataframes
coefficients = pd.concat([intercept_df, coefficients_df], axis=1)
coefficients

## Conclusion:

In [ ]:
The business should :
    -
    -
    -
    -